Conectar Python a Cassandra en Docker

In [36]:
from cassandra.cluster import Cluster
import time
import pandas as pd

cluster = Cluster(['localhost'], port=9042)

session = cluster.connect()

session.execute("""
CREATE KEYSPACE IF NOT EXISTS test_keyspace 
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

session.execute("USE test_keyspace")

session.execute("""
CREATE TABLE IF NOT EXISTS contactos (
    nombre text,
    direccion text,
    correo text PRIMARY KEY,
    telefono text,
    fecha_nacimiento date,
    empresa text,
    numero_tarjeta text,
    fecha_vencimiento text,
    codigo_postal text
)
""")

Carga de Datos en la Tabla

In [39]:
import csv

with open('../dataset_falso.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        session.execute("""
        INSERT INTO contactos (nombre, direccion, correo, telefono, fecha_nacimiento, empresa, numero_tarjeta, fecha_vencimiento, codigo_postal)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['Nombre'],
            row['Dirección'],
            row['Correo'],
            row['Teléfono'],
            row['Fecha de nacimiento'],
            row['Empresa'],
            row['Número de tarjeta'],
            row['Fecha de vencimiento'],
            row['Código postal']
        ))

Verificación de los Datos Cargados

In [40]:
rows = session.execute("SELECT * FROM contactos LIMIT 10")
for row in rows:
    print(row)

Row(correo='mgonzalez@example.com', codigo_postal='33787', direccion='C. Julieta Fonseca 66\nToledo, 30688', empresa='Promociones Internacionales S.L.U.', fecha_nacimiento=Date(-12424), fecha_vencimiento='05/26', nombre='Joaquín de Sáez', numero_tarjeta='502080583664', telefono='+34887 094 173')
Row(correo='ibatlle@example.com', codigo_postal='09203', direccion='Paseo de Carmelo Pablo 29\nCáceres, 06274', empresa='Compañía Canet & Asociados S.L.', fecha_nacimiento=Date(7563), fecha_vencimiento='07/30', nombre='Macario Martínez Folch', numero_tarjeta='4384066201407990869', telefono='+34874 833 207')
Row(correo='cantonmateo@example.com', codigo_postal='04691', direccion='Cañada de Maximiano Gimenez 49\nLugo, 41960', empresa='Cabrera y Ferrán S.Coop.', fecha_nacimiento=Date(-4837), fecha_vencimiento='11/32', nombre='Ileana Estrella Sanmiguel Mendoza', numero_tarjeta='568459799454', telefono='+34 876405142')
Row(correo='salvadorisidro@example.net', codigo_postal='22097', direccion='Calle J

Consultas de ejemplo

In [ ]:

# Definir las consultas CQL
queries = [
    "SELECT COUNT(*) FROM contactos;",  # Conteo de registros
    "SELECT * FROM contactos LIMIT 10;",  # Obtener los primeros 10 registros
    "SELECT * FROM contactos"  # Filtrado por correo
]

# Medir el tiempo de ejecución de cada consulta
for query in queries:
    print(f"\nEjecutando consulta: {query}")

    # Iniciar temporizador
    start_time = time.time()

    # Ejecutar consulta
    result = session.execute(query)
    result_list = list(result)
    # Detener temporizador
    end_time = time.time()

    # Mostrar algunos resultados y tiempo de ejecución
    print(f"Primeros resultados: {result_list[:5]}")  # Mostrar los primeros 5 resultados
    print(f"Tiempo de ejecución: {end_time - start_time} segundos\n")



Ejecutando consulta: SELECT COUNT(*) FROM contactos;
Primeros resultados: [Row(count=10041)]
Tiempo de ejecución: 0.3372969627380371 segundos


Ejecutando consulta: SELECT * FROM contactos LIMIT 10;
Primeros resultados: [Row(correo='mgonzalez@example.com', codigo_postal='33787', direccion='C. Julieta Fonseca 66\nToledo, 30688', empresa='Promociones Internacionales S.L.U.', fecha_nacimiento=Date(-12424), fecha_vencimiento='05/26', nombre='Joaquín de Sáez', numero_tarjeta='502080583664', telefono='+34887 094 173'), Row(correo='ibatlle@example.com', codigo_postal='09203', direccion='Paseo de Carmelo Pablo 29\nCáceres, 06274', empresa='Compañía Canet & Asociados S.L.', fecha_nacimiento=Date(7563), fecha_vencimiento='07/30', nombre='Macario Martínez Folch', numero_tarjeta='4384066201407990869', telefono='+34874 833 207'), Row(correo='cantonmateo@example.com', codigo_postal='04691', direccion='Cañada de Maximiano Gimenez 49\nLugo, 41960', empresa='Cabrera y Ferrán S.Coop.', fecha_nacimiento

In [ ]:
# Usar pandas para trabajar con los resultados de una consulta compleja
query = "SELECT * FROM contactos LIMIT 10;"
start_time = time.time()

# Ejecutar la consulta
result = session.execute(query)
data = list(result)

# Convertir los resultados a un DataFrame de Pandas
df = pd.DataFrame(data)

# Finalizar temporizador
end_time = time.time()
print(f"Consulta ejecutada en: {end_time - start_time} segundos")

# Mostrar el DataFrame con los primeros 5 registros
df.head()


Consulta ejecutada en: 0.005944728851318359 segundos


,correo,codigo_postal,direccion,empresa,fecha_nacimiento,fecha_vencimiento,nombre,numero_tarjeta,telefono
0,mgonzalez@example.com,33787,"C. Julieta Fonseca 66\nToledo, 30688",Promociones Internacionales S.L.U.,1935-12-27,05/26,Joaquín de Sáez,502080583664,+34887 094 173
1,ibatlle@example.com,09203,"Paseo de Carmelo Pablo 29\nCáceres, 06274",Compañía Canet & Asociados S.L.,1990-09-16,07/30,Macario Martínez Folch,4384066201407990869,+34874 833 207
2,cantonmateo@example.com,04691,"Cañada de Maximiano Gimenez 49\nLugo, 41960",Cabrera y Ferrán S.Coop.,1956-10-04,11/32,Ileana Estrella Sanmiguel Mendoza,568459799454,+34 876405142
3,salvadorisidro@example.net,22097,"Calle Juan Bautista Palau 99\nCuenca, 46971",Benitez y asociados S.Coop.,1938-05-06,06/29,Roldán Juliá-Salmerón,5293150424712493,+34 985 182 483
4,romanloreto@example.org,07843,"Pasaje de Nando Cabello 91 Apt. 82 \nMadrid, 0...",Gastón Ariza Bejarano S.A.,1940-03-07,09/28,Coral de Planas,6527357317083918,+34 820 564 697


In [ ]:
# Cerrar la conexión al finalizar
cluster.shutdown()